Load all the data into a duckdb database. Refer to MIMIC-III example for guidance to create your own database.

In [2]:
import duckdb

# establish connection
conn = duckdb.connect('mimic.db', read_only=False)

conn.sql('SHOW TABLES;')


┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ ADMISSIONS       │
│ DRGCODES         │
│ D_ICD_PROCEDURES │
│ ICUSTAYS         │
│ PATIENTS         │
│ PRESCRIPTIONS    │
│ PROCEDURES_ICD   │
└──────────────────┘

In [3]:
import pandas as pd

df = pd.read_csv('Data/ADMISSIONS.csv')
df.to_sql("ADMISSIONS", conn, index=False)

df = pd.read_csv('Data/D_ICD_PROCEDURES.csv')
df.to_sql("D_ICD_PROCEDURES", conn, index=False)

df = pd.read_csv('Data/DRGCODES.csv')
df.to_sql("DRGCODES", conn, index=False)

df = pd.read_csv('Data/ICUSTAYS.csv')
df.to_sql("ICUSTAYS", conn, index=False)

df = pd.read_csv('Data/PATIENTS.csv')
df.to_sql("PATIENTS", conn, index=False)

df = pd.read_csv('Data/PRESCRIPTIONS.csv')
df.to_sql("PRESCRIPTIONS", conn, index=False)

df = pd.read_csv('Data/PROCEDURES_ICD.csv')
df.to_sql("PROCEDURES_ICD", conn, index=False)
conn.sql('SHOW TABLES')

C:\Users\vDxbs\AppData\Local\Temp\ipykernel_12944\3038712933.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql("ADMISSIONS", conn, index=False)


ValueError: Table 'ADMISSIONS' already exists.

1) Create a summary of type of drugs and their total amount used by ethnicity. Report the top usage in each ethnicity group. You may have to make certain assumptions in calculating their total amount.

In [4]:
# prescriptions.drug & addmissions.ethnicity where subject_id and hadm_id are equal
# types are MAIN/BASE/ADDITIVE
query = conn.sql(
    """
    SELECT ADMISSIONS.ethnicity, 
           PRESCRIPTIONS.drug,
           count(*)
    FROM PRESCRIPTIONS
    JOIN ADMISSIONS
        ON PRESCRIPTIONS.subject_id = ADMISSIONS.subject_id
        AND PRESCRIPTIONS.hadm_id = ADMISSIONS.hadm_id
    GROUP BY ADMISSIONS.ethnicity, PRESCRIPTIONS.drug
    """
)
query


┌──────────────────┬─────────────────────────────┬──────────────┐
│    ethnicity     │            drug             │ count_star() │
│     varchar      │           varchar           │    int64     │
├──────────────────┼─────────────────────────────┼──────────────┤
│ WHITE            │ Senna                       │           66 │
│ WHITE            │ Aspirin                     │           44 │
│ WHITE            │ Tamsulosin                  │            5 │
│ WHITE            │ Docusate Sodium             │           67 │
│ WHITE            │ Sodium Chloride 0.9%  Flush │          151 │
│ WHITE            │ Albuterol Inhaler           │           14 │
│ WHITE            │ Azithromycin                │           16 │
│ WHITE            │ Oxycodone-Acetaminophen     │           18 │
│ WHITE            │ Creon 10                    │            2 │
│ WHITE            │ Ondansetron                 │           35 │
│   ·              │    ·                        │            · │
│   ·     

In [ ]:
df = query.df()

# AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN... ==== 5% Dextrose
# ASIAN ==== DSW
# BLACK?AFRICAN AMERICAN ==== Insuin
# HISPANIC OR LATINO ==== 5% Dextrose
# HISPANIC/LATINO - PUERTO RICAN ==== 0.9% Sodium Chloride
# OTHER ==== NS
# UNABLE TO OBTAIN ==== 0.9% Sodium Chloride
# UNKNOWN/NOT SPECIFIED ==== DSW
# WHITE ==== Potassium Chloride

sorted = df.sort_values(by=["ethnicity", "count_star()"], ascending=[True, False])
sorted.groupby('ethnicity').head(1)

,ethnicity,drug,count_star()
585,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,5% Dextrose,27
687,ASIAN,D5W,27
978,BLACK/AFRICAN AMERICAN,Insulin,38
1188,HISPANIC OR LATINO,5% Dextrose,28
846,HISPANIC/LATINO - PUERTO RICAN,0.9% Sodium Chloride,86
795,OTHER,NS,11
1251,UNABLE TO OBTAIN,0.9% Sodium Chloride,28
83,UNKNOWN/NOT SPECIFIED,D5W,37
319,WHITE,Potassium Chloride,381


2) Create a summary of procedures performed on patients by age groups (<=19, 20-49, 50-79, >80). Report the top three procedures, along with the name of the procedures, performed in each age group.

In [ ]:
# start by adding age column to addmissions
conn.sql("""
        ALTER TABLE ADMISSIONS DROP COLUMN age;
        ALTER TABLE ADMISSIONS ADD COLUMN age INT;
        """)

conn.sql("""
        UPDATE ADMISSIONS
        SET age = LEAST((CAST(ADMISSIONS.admittime as date) - CAST(PATIENTS.dob as date))/365.25, 89)
        FROM PATIENTS
        WHERE ADMISSIONS.subject_id = PATIENTS.subject_id
        """)

conn.sql("""
        SELECT age
        FROM admissions
        """)


In [60]:
# <= 19 query
df = conn.sql(
    """
    SELECT D_ICD_PROCEDURES.short_title,
           count(*)
    FROM D_ICD_PROCEDURES
    JOIN PROCEDURES_ICD
        ON PROCEDURES_ICD.icd9_code = D_ICD_PROCEDURES.icd9_code
        JOIN ADMISSIONS
            ON PROCEDURES_ICD.subject_id = ADMISSIONS.subject_id
            AND PROCEDURES_ICD.hadm_id = ADMISSIONS.hadm_id
            AND ADMISSIONS.age <= 19
    GROUP BY D_ICD_PROCEDURES.short_title
    """
).df()
df.sort_values('count_star()', ascending=False).head(3)

,short_title,count_star()
4,Venous cath NEC,2
1,Cl fx reduc-femur,1
0,Incision of lung,1


In [62]:
# 20-49 query
df = conn.sql(
    """
    SELECT D_ICD_PROCEDURES.short_title,
           count(*)
    FROM D_ICD_PROCEDURES
    JOIN PROCEDURES_ICD
        ON PROCEDURES_ICD.icd9_code = D_ICD_PROCEDURES.icd9_code
        JOIN ADMISSIONS
            ON PROCEDURES_ICD.subject_id = ADMISSIONS.subject_id
            AND PROCEDURES_ICD.hadm_id = ADMISSIONS.hadm_id
            AND ADMISSIONS.age >= 20
            AND ADMISSIONS.age <= 49
    GROUP BY D_ICD_PROCEDURES.short_title
    """
).df()
df.sort_values('count_star()', ascending=False).head(3)

,short_title,count_star()
16,Venous cath NEC,9
23,Entral infus nutrit sub,7
4,Percu abdominal drainage,6


In [63]:
# 50-79 query
df = conn.sql(
    """
    SELECT D_ICD_PROCEDURES.short_title,
           count(*)
    FROM D_ICD_PROCEDURES
    JOIN PROCEDURES_ICD
        ON PROCEDURES_ICD.icd9_code = D_ICD_PROCEDURES.icd9_code
        JOIN ADMISSIONS
            ON PROCEDURES_ICD.subject_id = ADMISSIONS.subject_id
            AND PROCEDURES_ICD.hadm_id = ADMISSIONS.hadm_id
            AND ADMISSIONS.age >= 50
            AND ADMISSIONS.age <= 79
    GROUP BY D_ICD_PROCEDURES.short_title
    """
).df()
df.sort_values('count_star()', ascending=False).head(3)

,short_title,count_star()
27,Venous cath NEC,25
46,Entral infus nutrit sub,22
18,Packed cell transfusion,13


In [64]:
# >= 80 query
df = conn.sql(
    """
    SELECT D_ICD_PROCEDURES.short_title,
           count(*)
    FROM D_ICD_PROCEDURES
    JOIN PROCEDURES_ICD
        ON PROCEDURES_ICD.icd9_code = D_ICD_PROCEDURES.icd9_code
        JOIN ADMISSIONS
            ON PROCEDURES_ICD.subject_id = ADMISSIONS.subject_id
            AND PROCEDURES_ICD.hadm_id = ADMISSIONS.hadm_id
            AND ADMISSIONS.age >= 80
    GROUP BY D_ICD_PROCEDURES.short_title
    """
).df()
df.sort_values('count_star()', ascending=False).head(3)

,short_title,count_star()
20,Venous cath NEC,20
10,Packed cell transfusion,13
49,Insert endotracheal tube,8


3) How long do patients stay in the ICU? Is there a difference in the ICU length of stay among gender or ethnicity?

In [ ]:
# create column to take days in ICU
# start by adding days column to addmissions
conn.sql("""
        ALTER TABLE ICUSTAYS DROP COLUMN days;
        ALTER TABLE ICUSTAYS ADD COLUMN days INT;
        """)

conn.sql("""
        UPDATE ICUSTAYS
        SET days = CAST(ICUSTAYS.outtime as date) - CAST(ICUSTAYS.intime as date)
        """)

conn.sql("""
        SELECT days
        FROM ICUSTAYS
        """)

In [79]:
# now lets take the days in icu and group by gender
conn.sql("""
        SELECT PATIENTS.gender,
              AVG(ICUSTAYS.days) AS average_stay_in_days
         FROM ICUSTAYS
         JOIN PATIENTS
              ON ICUSTAYS.subject_id = PATIENTS.subject_id
         GROUP BY PATIENTS.gender
         """)

┌─────────┬──────────────────────┐
│ gender  │ average_stay_in_days │
│ varchar │        double        │
├─────────┼──────────────────────┤
│ F       │    5.476190476190476 │
│ M       │   3.5205479452054793 │
└─────────┴──────────────────────┘

In [81]:
# now lets take the days in icu and group by ethnicity
conn.sql("""
        SELECT ADMISSIONS.ethnicity,
              AVG(ICUSTAYS.days) AS average_stay_in_days
         FROM ICUSTAYS
         JOIN ADMISSIONS
              ON ICUSTAYS.subject_id = ADMISSIONS.subject_id
         GROUP BY ADMISSIONS.ethnicity
         """)

┌──────────────────────────────────────────────────────────┬──────────────────────┐
│                        ethnicity                         │ average_stay_in_days │
│                         varchar                          │        double        │
├──────────────────────────────────────────────────────────┼──────────────────────┤
│ UNKNOWN/NOT SPECIFIED                                    │    4.461538461538462 │
│ ASIAN                                                    │                  4.0 │
│ BLACK/AFRICAN AMERICAN                                   │    6.888888888888889 │
│ OTHER                                                    │                  1.0 │
│ HISPANIC OR LATINO                                       │    7.333333333333333 │
│ UNABLE TO OBTAIN                                         │                 14.0 │
│ AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE │                 11.5 │
│ WHITE                                                    │    4.0245901639

In [82]:
conn.close()

In [ ]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

# ssl setup
ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('/home/ubuntu/sf-class2-root.crt')  # change your file path for locating the certificate
ssl_context.verify_mode = CERT_REQUIRED

# boto3 session setup
boto_session = boto3.Session(region_name="us-east-2")  # this AWS credentials is specific to `us-east-2` region

In [ ]:
# authorization setup with SigV4
auth_provider = SigV4AuthProvider(boto_session)

In [ ]:
#cluster setup 
cluster = Cluster(['cassandra.us-east-2.amazonaws.com'], 
                  ssl_context=ssl_context, 
                  auth_provider=auth_provider, 
                  port=9142)  # TLS only communicates on port 9142

In [ ]:
# establishing connection to Keyspace
session = cluster.connect()
# Insert any CQL queries between .connect() and .shutdown()

In [ ]:
# For example, show all keyspaces created
r = session.execute('''
    SELECT * FROM system_schema.keyspaces;
    ''')
print(r.current_rows)

In [ ]:
# For example, create a keyspace for HW2
r = session.execute('''
    CREATE KEYSPACE IF NOT EXISTS csn4634_hw2 
    WITH replication = {'class': 'SingleRegionStrategy'};
    ''')
print(r.current_rows)

In [ ]:
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra import ConsistencyLevel


# Define execution profile with LOCAL_QUORUM
execution_profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)

# Cluster setup with correct profile
cluster = Cluster(
    ['cassandra.us-east-2.amazonaws.com'],
    ssl_context=ssl_context,
    auth_provider=auth_provider,
    port=9142,
    execution_profiles={EXEC_PROFILE_DEFAULT: execution_profile}
)

# establishing connection to Keyspace
session = cluster.connect()
session.set_keyspace('csn4634_hw2')  

In [ ]:
# creating table
session.execute("""
CREATE TABLE IF NOT EXISTS q1 (
    ethnicity TEXT PRIMARY KEY,
    drug TEXT,
    count INT,
);
""")

In [ ]:
# time to fill table
import pandas as pd
pres = pd.read_csv("Data/PRESCRIPTIONS.csv")
ads = pd.read_csv("Data/ADMISSIONS.csv")

combo = pd.merge(pres, ads[['subject_id', 'hadm_id', 'ethnicity']], on=['subject_id', 'hadm_id'], how = 'inner')
counts = combo.groupby(['ethnicity', 'drug']).size().reset_index(name = 'count')
top_counts = counts.sort_values(['ethnicity','count'], ascending = [True,False]).groupby('ethnicity').first().reset_index()

#insert row by row
for _, row in top_counts.iterrows():
    session.execute(
        """
        INSERT INTO q1 (ethnicity, drug, count)
        VALUES (%s, %s, %s)
        """,
        (row['ethnicity'], row['drug'], int(row['count']))
    )


In [ ]:
# now query
ethnicities =session.execute(""" SELECT * FROM q1 """)
for ethnicity in ethnicities:
    print("The most common drug for those of ethnicity {" + ethnicity.ethnicity + "} is {" + ethnicity.drug + "}")

In [ ]:
# create table
session.set_keyspace('csn4634_hw2')
session.execute("""
CREATE TABLE IF NOT EXISTS q2 (
    age_range TEXT,
    icd9_code TEXT,
    count INT,
    PRIMARY KEY (age_range, icd9_code)
);
""")

In [ ]:
# add age to admissions
import pandas as pd

admissions = pd.read_csv('Data/ADMISSIONS.csv')
patients = pd.read_csv('Data/PATIENTS.csv')
d_icd_procedures = pd.read_csv('Data/D_ICD_PROCEDURES.csv')
procedures_icd = pd.read_csv('Data/PROCEDURES_ICD.csv')

admissions = admissions.merge(patients[['subject_id', 'dob']], on='subject_id', how='left')
def getYear(x):
    x = pd.to_datetime(x)
    return x.year
admissions['age'] = admissions['admittime'].apply(getYear) - admissions['dob'].apply(getYear)



In [ ]:
def categorize_age(age):
    if age <= 19:
        return '<=19'
    elif 20 <= age <= 49:
        return '20-49'
    elif 50 <= age <= 79:
        return '50-79'
    else:
        return '>=80'

admissions['age_range'] = admissions['age'].apply(categorize_age)

admissions_procedures = admissions.merge(procedures_icd[['subject_id', 'icd9_code', 'hadm_id']], on=['subject_id', 'hadm_id'], how='left')

age_group_procedure_count = admissions_procedures.groupby(['age_range', 'icd9_code']).size().reset_index(name='count')

top_procedures = age_group_procedure_count.groupby('age_range').apply(lambda x: x.nlargest(3, 'count')).reset_index(drop=True)
top_procedures['icd9_code'] = top_procedures['icd9_code'].apply(lambda x: str(int(x)) if isinstance(x, float) else str(x))


insert_query = """
    INSERT INTO q2 (age_range, icd9_code, count)
    VALUES (%s, %s, %s)
"""
session.set_keyspace('csn4634_hw2')

for index, row in top_procedures.iterrows():
    session.execute(insert_query, (row['age_range'], row['icd9_code'], row['count']))


In [ ]:
# Load the d_icd_procedures.csv to get procedure names
d_icd_procedures = pd.read_csv('Data/D_ICD_PROCEDURES.csv')

# Make sure codes are strings and clean
d_icd_procedures['icd9_code'] = d_icd_procedures['icd9_code'].astype(str).str.strip()

# Build a dictionary for fast lookup
procedure_dict = dict(zip(d_icd_procedures['icd9_code'], d_icd_procedures['short_title']))


age_ranges = ['<=19', '20-49', '50-79', '>=80']

for age_range in age_ranges:
    result = session.execute("""
        SELECT icd9_code, count
        FROM q2
        WHERE age_range = %s
    """, (age_range,))

    
    # Sort the results by 'count' to get the top 3 procedures
    sorted_result = sorted(result, key=lambda row: row.count, reverse=True)[:3]
    
    print(f"Top 3 Procedures for Age Range {age_range}:")
    for row in sorted_result:
        icd9_code = row.icd9_code
        procedure_name = d_icd_procedures[d_icd_procedures['icd9_code'] == icd9_code]['short_title'].iloc[0]
        print(f"ICD9 Code: {icd9_code}, Procedure Name: {procedure_name}, Count: {row.count}")
    print()


In [ ]:
#make table
session.execute("DROP TABLE IF EXISTS q3")

session.execute("""
CREATE TABLE IF NOT EXISTS q3(
    demographic TEXT,
    time_in_icu DOUBLE,
    PRIMARY KEY (demographic)
);
""")

In [ ]:
# fill table
icustays = pd.read_csv('Data/ICUSTAYS.csv')
patients = pd.read_csv('Data/PATIENTS.csv')

merged = icustays.merge(patients[['subject_id', 'gender']], on='subject_id', how='left')
merged['intime'] = pd.to_datetime(merged['intime'])
merged['outtime'] = pd.to_datetime(merged['outtime'])

merged['time_in_icu'] = (merged['outtime'] - merged['intime']).dt.days

insert = """
    INSERT INTO q3 (demographic, time_in_icu)
    VALUES (%s, %s)
"""
per_gender = merged.groupby('gender')['time_in_icu'].mean()

session.execute(insert, ('M', per_gender[0]))
session.execute(insert, ('F', per_gender[1]))

admissions = pd.read_csv('Data/ADMISSIONS.csv')

merged = icustays.merge(admissions[['subject_id', 'ethnicity']], on='subject_id', how='left')
merged['intime'] = pd.to_datetime(merged['intime'])
merged['outtime'] = pd.to_datetime(merged['outtime'])

merged['time_in_icu'] = (merged['outtime'] - merged['intime']).dt.days

per_ethnicity = merged.groupby('ethnicity')['time_in_icu'].mean()
i = 0
for ethnic in merged['ethnicity'].unique():
    session.execute(insert, (ethnic, per_ethnicity[i]))
    i += 1

In [ ]:
#query
demographics = set(merged['ethnicity'].unique()) | ( set(patients['gender'].unique()))
for demo in demographics:
    query = "SELECT time_in_icu FROM q3 WHERE demographic = %s"
    result = session.execute(query, (demo,))
    for row in result:
        print(f"Mean ICU time for {demo} is {row.time_in_icu}")

In [ ]:
session.shutdown()
cluster.shutdown()
